# The dataset conversion is just for instance segmentation and object detection. Classification and segmentation can be used as given by 1.5

In [ ]:
import json
import cv2
import numpy as np
import os
import shutil

In [ ]:
output_dir = "finaltestdataset"
segmentation_dataset_train_path = r"C:\Users\philmarq\source\repos\Daten\neuestes\dataset\dataset/train"
segmentation_dataset_valid_path = r"C:\Users\philmarq\source\repos\Daten\neuestes\dataset\dataset\valid"

output_images_dir = os.path.join(output_dir, "images")
os.makedirs(output_images_dir, exist_ok=True)


In [ ]:
# Function to copy images from source to destination
def copy_images(source_dir, dest_dir):
    # Get all files in the source directory
    for root, _, files in os.walk(source_dir):
        for file in files:
            # Check if file is an image
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.tif')):
                source_file = os.path.join(root, file)
                dest_file = os.path.join(dest_dir, file)
                shutil.copy2(source_file, dest_file)
# convert to coco format


#nc will be automatically identified
def create_files( folder, nc=1, single_class=False):
        """
        Create coco files
        
        """
        files = {}
        files["info"] = {
            "year": 2222,
            "version": "1.0",
            "description": "Object detection",
            "date_created": "2222",
        }
        files["licenses"] = [
            {
                "id": 1,
                "name": "GNU General Public License v3.0",
                "url": "https://github.com/zhiqwang/yolov5-rt-stack/blob/master/LICENSE",
            }
        ]
        files["type"] = "instances"
        files["categories"] = []
        files["annotations"] = []
        files["images"] = []

        start = os.path.join(folder, "labels")
        all_files = os.listdir(start)
        if single_class:
            nc = 1
        max_val = 0
        for cnt_out, img in enumerate(os.listdir(start)):
            im = cv2.imread(os.path.join(start, img), 0)
            max_val = max(max_val, np.max(im))
        nc = max_val
        print(nc)
        for i in range(nc):
            files["categories"].append(
                {"id": i, "name": str(i), "supercategory": str(i)}
            )
        all_annos = 0
        for cnt_out, img in enumerate(os.listdir(start)):
            im = cv2.imread(os.path.join(start, img), 0)
            files["images"].append(
                {
                    "date_captured": "2021",
                    "file_name": img,
                    "id": cnt_out,
                    "height": im.shape[0],
                    "width": im.shape[1],
                }
            )

            for i in range(0, nc):
                im = cv2.imread(os.path.join(start, img), 0)
                tmp = im.copy()
                if not single_class:
                    tmp[tmp != (i + 1)] = 0
                    tmp[tmp == (i + 1)] = 255
                else:
                    tmp[tmp > 0] = 255

                contours, hierachy = cv2.findContours(
                    tmp, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
                )
                blank = np.zeros_like(tmp)

                for cnt, cont in enumerate(contours):
                    segmentation = []
                    xmin, ymin, width, height = cv2.boundingRect(cont)  # bounding box
                    if width * height < 3:
                        continue
                    image_height = im.shape[0]
                    image_width = im.shape[1]
                    cont = (
                        cont.flatten().tolist()
                    )  # contour as 1d array has shape (x1,y1,x2,y2,...,x_n, y_n)
                    # https://github.com/facebookresearch/Detectron/issues/100#issuecomment-362882830
                    if len(cont) > 4:  # only of at least 2 points are there
                        segmentation.append(cont)
                    else:
                        continue
                    if (
                        len(segmentation) == 0
                    ):  # check again if segmentations are in list
                        continue

                    files["annotations"].append(
                        {
                            "segmentation": segmentation,
                            "area": width * height,
                            "image_id": cnt_out,
                            "iscrowd": 0,
                            "bbox": [xmin, ymin, width, height],
                            "category_id": i,
                            "id": all_annos,
                        }
                    )
                    all_annos += 1

        return files
    

In [ ]:
# Copy images from both train and valid directories to the output images directory
copy_images(os.path.join(segmentation_dataset_train_path, "images"), output_images_dir)
copy_images(os.path.join(segmentation_dataset_valid_path, "images"), output_images_dir)
train_json = create_files(segmentation_dataset_train_path)
valid_json = create_files(segmentation_dataset_valid_path)

In [ ]:
with open(os.path.join(output_dir,"train.json"), 'w') as json_file:
    json.dump(train_json, json_file)

with open(os.path.join(output_dir, "valid.json"), 'w') as json_file:
    json.dump(valid_json, json_file)

# Now actually train the models

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # select gpu
from mmwrapper.src.api import get_runner

In [ ]:
#runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\object_detection\object_detection.yaml")
#runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\segmentation\segmentation.yaml")
runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\instance_segmentation\instance_seg.yaml")
#runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\classification\classification.yaml")

In [ ]:
runner.train()

In [ ]:
pip install future tensorboard